In [6]:
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex
import numpy as np
import json
import pandas as pd
from scipy import stats
import os

# Experiments and Functions

In [7]:
experiments = [
            {'id': '1', 'source':'imdb', 'target':'uwcse', 'predicate':'workedunder', 'to_predicate':'advisedby', 'arity': 2},
            #{'id': '2', 'source':'uwcse', 'target':'imdb', 'predicate':'advisedby', 'to_predicate':'workedunder', 'arity': 2},
            {'id': '3', 'source':'imdb', 'target':'cora', 'predicate':'workedunder', 'to_predicate':'samevenue', 'arity': 2},
            {'id': '4', 'source':'cora', 'target':'imdb', 'predicate':'samevenue', 'to_predicate':'workedunder', 'arity': 2},
            ##{'id': '5', 'source':'uwcse', 'target':'cora', 'predicate':'advisedby', 'to_predicate':'samevenue', 'arity': 2},
            ##{'id': '6', 'source':'cora', 'target':'uwcse', 'predicate':'samevenue', 'to_predicate':'advisedby', 'arity': 2},
            {'id': '7', 'source':'yeast', 'target':'twitter', 'predicate':'proteinclass', 'to_predicate':'accounttype', 'arity': 2},
            {'id': '8', 'source':'twitter', 'target':'yeast', 'predicate':'accounttype', 'to_predicate':'proteinclass', 'arity': 2},
            {'id': '9', 'source':'nell_sports', 'target':'nell_finances', 'predicate':'teamplayssport', 'to_predicate':'companyeconomicsector', 'arity': 2},
            #{'id': '10', 'source':'nell_finances', 'target':'nell_sports', 'predicate':'companyeconomicsector', 'to_predicate':'teamplayssport', 'arity': 2}
]

In [8]:
def load_data(experiments, metric, directory):
    data = {}
    for item in experiments:
        experiment_title = item['id'] + '_' + item['source'] + '_' + item['target']
        
        if(metric != ''):
            file_path = os.getcwd() + directory + experiment_title + '/' + experiment_title + '_fasttext_{}.json'.format(metric)
        else:
            file_path = os.getcwd() + directory + experiment_title + '/' + experiment_title + '.json'
            
        #if(os.path.isfile(file_path)):
        with open(file_path, 'r') as fp:
            results = json.load(fp)

            data[experiment_title] = results
    return data

# Estudo dos Mapeamentos

## Depth-First Mapping

In [9]:
# Setting paths parameters
mapping_order = 'deep'
k = 1
compare_stopwords = False

#Removing Yeast and Twitter pair of experiments because there is no stopwords in predicates
#exp = experiments.copy()
#exp.remove({'id': '7', 'source':'yeast', 'target':'twitter', 'predicate':'proteinclass', 'to_predicate':'accounttype', 'arity': 2})
#exp.remove({'id': '8', 'source':'twitter', 'target':'yeast', 'predicate':'accounttype', 'to_predicate':'proteinclass', 'arity': 2})

In [16]:
path = f'/results/transfer/{mapping_order}/w_stopwords/k_{k}/'
data_softcosine = load_data(experiments, 'softcosine', path)
data_euclidean = load_data(experiments, 'euclidean', path)
data_wmd = load_data(experiments, 'wmd', path)
data_rwmd = load_data(experiments, 'relax-wmd', path)

baseline_path = f'/results/baselines/transfer/'
baseline_data = load_data(experiments, '', baseline_path)

for j in range(len(experiments)):
    dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
    #if dataset in data['results']: 
    display(Markdown('# Results for ' + dataset))
    table = []
    for metric in ['AUC ROC', 'AUC PR', 'CLL']: #, 'Learning and Revision time', 'Inference time']:
        display(Markdown('## ' + metric))
        
        trans_softcosine = np.zeros(len(data_softcosine[str(dataset)][0]))
        trans_softcosine_no_rev = np.zeros(len(data_softcosine[str(dataset)][0]))
        
        trans_euclidean = np.zeros(len(data_euclidean[str(dataset)][0]))
        trans_euclidean_no_rev = np.zeros(len(data_euclidean[str(dataset)][0]))
        
        trans_wmd = np.zeros(len(data_wmd[str(dataset)][0]))
        trans_wmd_no_rev = np.zeros(len(data_wmd[str(dataset)][0]))
        
        trans_relax_wmd = np.zeros(len(data_rwmd[str(dataset)][0]))
        trans_relax_wmd_no_rev = np.zeros(len(data_rwmd[str(dataset)][0]))
        
        tree_no_rev = np.zeros(len(baseline_data[str(dataset)][0]))
        tree = np.zeros(len(baseline_data[str(dataset)][0]))
        
        rdnb_no_rev = np.zeros(len(baseline_data[str(dataset)][0]))
        rdnb = np.zeros(len(baseline_data[str(dataset)][0]))
        
        #for m in range(len(treeboostler_data[str(dataset)][0])):
        m = 0
        trans_softcosine += np.array([item['transfer'][metric] for item in data_softcosine[str(dataset)][m]])
        trans_softcosine_no_rev += np.array([item['transfer']['parameter'][metric] for item in data_softcosine[str(dataset)][m]])

        trans_euclidean += np.array([item['transfer'][metric] for item in data_euclidean[str(dataset)][m]])
        trans_euclidean_no_rev += np.array([item['transfer']['parameter'][metric] for item in data_euclidean[str(dataset)][m]])

        trans_wmd += np.array([item['transfer'][metric] for item in data_wmd[str(dataset)][m]])
        trans_wmd_no_rev += np.array([item['transfer']['parameter'][metric] for item in data_wmd[str(dataset)][m]])
        
        trans_relax_wmd += np.array([item['transfer'][metric] for item in data_rwmd[str(dataset)][m]])
        trans_relax_wmd_no_rev += np.array([item['transfer']['parameter'][metric] for item in data_rwmd[str(dataset)][m]])

        tree_no_rev += np.array([item['transfer']['parameter'][metric] for item in baseline_data[str(dataset)][m]])
        tree += np.array([item['transfer'][metric] for item in baseline_data[str(dataset)][m]])
        
        # Fora do FOR

        rdnb_no_rev += np.array([item['rdn_b'][metric] for item in baseline_data[str(dataset)][0]]) 
        rdnb += np.array([item['rdn_b'][metric] for item in baseline_data[str(dataset)][0]])
            
        #trans_softcosine /= len(data_softcosine[str(dataset)])
        #trans_softcosine_no_rev /= len(data_softcosine_no_rev[str(dataset)])
        
        #trans_euclidean /= len(data_euclidean[str(dataset)])
        #trans_euclidean_no_rev /= len(data_euclidean_no_rev[str(dataset)])
        
        #trans_wmd /= len(data_wmd[str(dataset)])
        #trans_wmd_no_rev /= len(data_wmd_no_rev[str(dataset)])
        
        #tree /= len(treeboostler_data[str(dataset)])
        #tree_no_rev /= len(treeboostler_data[str(dataset)])
        
        #rdnb /= len(rdnb_data[str(dataset)])
        #rdnb_no_rev /= len(rdnb_data[str(dataset)])
        
        pvalue_table = []
        tvalue, pvalue = stats.ttest_rel(trans_softcosine,tree)
        tvalue2, pvalue2 = stats.ttest_rel(trans_softcosine, rdnb)
        pvalue_table.append(['TransBoostler Soft Cosine', '%.3f' % (pvalue), '%.3f' % (pvalue2)])
        del pvalue,pvalue2
        
        tvalue, pvalue = stats.ttest_rel(trans_euclidean,tree)
        tvalue2, pvalue2 = stats.ttest_rel(trans_euclidean,rdnb)

        pvalue_table.append(['TransBoostler Euclidean', '%.3f' % (pvalue), '%.3f' % (pvalue2)])
        del pvalue,pvalue2
        
        tvalue, pvalue = stats.ttest_rel(trans_wmd,tree)
        tvalue2, pvalue2 = stats.ttest_rel(trans_wmd,rdnb)
        
        pvalue_table.append(['TransBoostler WMD', '%.3f' % (pvalue), '%.3f' % (pvalue2)])
        del pvalue,pvalue2
        
        # No Revision
        
        tvalue, pvalue = stats.ttest_rel(trans_softcosine_no_rev,tree_no_rev)
        tvalue2, pvalue2 = stats.ttest_rel(trans_softcosine_no_rev,rdnb_no_rev)
        
        pvalue_table.append(['TransBoostler* Soft Cosine', '%.3f' % (pvalue), '%.3f' % (pvalue2)])
        del pvalue,pvalue2
        
        tvalue, pvalue = stats.ttest_rel(trans_euclidean_no_rev,tree_no_rev)
        tavlue2, pvalue2 = stats.ttest_rel(trans_euclidean_no_rev,rdnb_no_rev)
        
        pvalue_table.append(['TransBoostler* Euclidean', '%.3f' % (pvalue), '%.3f' % (pvalue2)])
        del pvalue,pvalue2
        
        tvalue, pvalue = stats.ttest_rel(trans_wmd_no_rev,tree_no_rev)
        tvalue, pvalue2 = stats.ttest_rel(trans_wmd_no_rev,rdnb_no_rev)
        
        pvalue_table.append(['TransBoostler* WMD', '%.3f' % (pvalue), '%.3f' % (pvalue2)])
        del pvalue,pvalue2
        
        display(pd.DataFrame(pvalue_table, columns=['p-value', 'TreeBoostler', 'RDN-B']))
        print('TransBoostler Soft Cosine', ','.join(str(v) for v in trans_softcosine))
        
        print('TransBoostler Euclidean', ','.join(str(v) for v in trans_euclidean))
        
        print('TransBoostler WMD', ','.join(str(v) for v in trans_wmd))
        
        print('TreeBoostler', ','.join(str(v) for v in tree))
        
        print('TransBoostler* Soft Cosine', ','.join(str(v) for v in trans_softcosine_no_rev))
        print('TransBoostler* Euclidean', ','.join(str(v) for v in trans_euclidean_no_rev))
        print('TransBoostler* WMD', ','.join(str(v) for v in trans_wmd_no_rev))
        
        print('TreeBoostler*', ','.join(str(v) for v in tree_no_rev))
        
        print('RDN-B', ','.join(str(v) for v in rdnb))
       

# Results for 1_imdb_uwcse

## AUC ROC

,p-value,TreeBoostler,RDN-B
0,TransBoostler Soft Cosine,0.580,0.853
1,TransBoostler Euclidean,0.328,0.058
2,TransBoostler WMD,0.654,0.461
3,TransBoostler* Soft Cosine,0.000,0.000
4,TransBoostler* Euclidean,0.374,0.846
5,TransBoostler* WMD,0.016,0.012


TransBoostler Soft Cosine 0.945976,0.94275,0.907476,0.955596,0.924661
TransBoostler Euclidean 0.951531,0.943754,0.937167,0.954448,0.947994
TransBoostler WMD 0.952244,0.940502,0.904213,0.944886,0.942951
TreeBoostler 0.935982,0.947314,0.906262,0.953218,0.953771
TransBoostler* Soft Cosine 0.609051,0.608882,0.608826,0.608066,0.607124
TransBoostler* Euclidean 0.935982,0.932863,0.937167,0.936897,0.937621
TransBoostler* WMD 0.908168,0.903558,0.904213,0.907372,0.905865
TreeBoostler* 0.935982,0.932863,0.904213,0.936897,0.937621
RDN-B 0.940612,0.942783,0.9075,0.945827,0.936214


## AUC PR

...

,p-value,TreeBoostler,RDN-B
0,TransBoostler Soft Cosine,0.413,0.674
1,TransBoostler Euclidean,0.430,0.812
2,TransBoostler WMD,0.124,0.395
3,TransBoostler* Soft Cosine,0.004,0.007
4,TransBoostler* Euclidean,0.004,0.007
5,TransBoostler* WMD,0.004,0.007


TransBoostler Soft Cosine 0.097892,0.09501,0.068899
TransBoostler Euclidean 0.095768,0.101298,0.071706
TransBoostler WMD 0.098802,0.148008,0.106665
TreeBoostler 0.088366,0.11225,0.092273
TransBoostler* Soft Cosine 0.001609,0.001568,0.001571
TransBoostler* Euclidean 0.001617,0.001609,0.001661
TransBoostler* WMD 0.001537,0.001528,0.001513
TreeBoostler* 0.09329,0.11225,0.092273
RDN-B 0.105361,0.078953,0.095584


## CLL

,p-value,TreeBoostler,RDN-B
0,TransBoostler Soft Cosine,0.095,0.036
1,TransBoostler Euclidean,0.212,0.069
2,TransBoostler WMD,0.010,0.073
3,TransBoostler* Soft Cosine,0.002,0.001
4,TransBoostler* Euclidean,0.007,0.002
5,TransBoostler* WMD,0.009,0.002


TransBoostler Soft Cosine -0.317487,-0.322892,-0.334831
TransBoostler Euclidean -0.310794,-0.326692,-0.313707
TransBoostler WMD -0.313065,-0.306901,-0.306706
TreeBoostler -0.309248,-0.302375,-0.30355
TransBoostler* Soft Cosine -0.370435,-0.363133,-0.370096
TransBoostler* Euclidean -0.363284,-0.370425,-0.360858
TransBoostler* WMD -0.35837,-0.366879,-0.358195
TreeBoostler* -0.312329,-0.302375,-0.30355
RDN-B -0.2995,-0.30063,-0.300652
